In [14]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [15]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch
from operator import itemgetter

In [16]:
llm = ChatOpenAI(
    api_key=key, 
    model_name='gpt-4o-mini',
    temperature=0.1
)

In [17]:
output_parser = StrOutputParser()

In [18]:
prompt = PromptTemplate.from_template(
    """주어진 사용자 질문을 `수학`, `과학`, 또는 `기타` 중 하나로 분류하세요. 한 단어 이상으로 응답하지 마세요.

<question>
{question}
</question>

Classification:"""
)

In [19]:
# 체인
chain = prompt | llm | output_parser

In [20]:
math_chain = (
    PromptTemplate.from_template(
        """You are an expert in math. \
Always answer questions starting with "깨봉선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI(model="gpt-4o-mini")
)

In [21]:
science_chain = (
    PromptTemplate.from_template(
        """You are an expert in science. \
Always answer questions starting with "아이작 뉴턴 선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI(model="gpt-4o-mini")
)

In [22]:
general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question concisely:

Question: {question}
Answer:"""
    )
    | ChatOpenAI(model="gpt-4o-mini")
)

In [23]:
branch = RunnableBranch(
    # 주제에 "수학"이 포함되어 있는지 확인하고, 포함되어 있다면 math_chain을 실행합니다.
    (lambda x: "수학" in x["topic"].lower(), math_chain),
    # 주제에 "과학"이 포함되어 있는지 확인하고, 포함되어 있다면 science_chain을 실행합니다.
    (lambda x: "과학" in x["topic"].lower(), science_chain),
    # 위의 조건에 해당하지 않는 경우 general_chain을 실행합니다.
    general_chain,
)

In [24]:
# 주제와 질문을 입력받아 branch를 실행하는 전체 체인을 정의합니다.
full_chain = (
    {"topic": chain, "question": itemgetter("question")} | branch | StrOutputParser()
)

In [25]:
# 질문을 입력하여 전체 체인을 실행합니다.
full_chain.invoke({"question": "미적분의 개념에 대해 말씀해 주세요."})

'깨봉선생님께서 말씀하시기를, 미적분은 수학의 한 분야로, 두 가지 주요 개념인 미분과 적분으로 구성되어 있습니다. 미분은 함수의 변화율을 다루며, 특정 점에서의 기울기나 함수의 순간적인 변화를 측정합니다. 반면, 적분은 함수의 넓이나 총합을 구하는 과정으로, 주어진 구간에서 함수의 값을 모두 더하는 것과 관련이 있습니다. \n\n미적분은 물리학, 공학, 경제학 등 다양한 분야에서 중요한 역할을 하며, 변화하는 현상을 수학적으로 모델링하고 분석하는 데 필수적인 도구입니다. 이 두 개념은 서로 밀접하게 연결되어 있으며, 미적분학의 기본 정리에서는 미분과 적분이 서로 반대의 과정임을 보여줍니다. 이를 통해 우리는 복잡한 문제를 해결하고 다양한 현상을 이해할 수 있습니다.'

In [26]:
# 질문을 입력하여 전체 체인을 실행합니다.
full_chain.invoke({"question": "중력 가속도는 어떻게 계산하나요?"})

'아이작 뉴턴 선생님께서 말씀하시기를, 중력 가속도는 지구의 표면 근처에서 물체가 받는 중력의 영향을 나타내는 값으로, 일반적으로 \\( g \\)로 표기됩니다. 이 값은 약 \\( 9.81 \\, \\text{m/s}^2 \\)로 알려져 있습니다. \n\n중력 가속도를 계산하기 위해서는 만유인력 법칙을 사용할 수 있습니다. 만유인력 법칙에 따르면, 두 물체 간의 중력 \\( F \\)는 다음과 같이 표현됩니다:\n\n\\[\nF = G \\frac{m_1 m_2}{r^2}\n\\]\n\n여기서 \\( G \\)는 만유인력 상수, \\( m_1 \\)과 \\( m_2 \\)는 두 물체의 질량, \\( r \\)은 두 물체 간의 거리입니다. 지구의 경우, 예를 들어 물체의 질량을 \\( m \\)로 하고, 지구의 질량을 \\( M \\), 지구의 반지름을 \\( R \\)로 설정하면, 중력 가속도 \\( g \\)는 다음과 같이 표현됩니다:\n\n\\[\ng = \\frac{F}{m} = G \\frac{M}{R^2}\n\\]\n\n이 식을 통해 지구의 질량과 반지름을 알고 있다면 중력 가속도를 계산할 수 있습니다.'

In [27]:
# 질문을 입력하여 전체 체인을 실행합니다.
full_chain.invoke({"question": "RAG(Retrieval Augmented Generation)은 무엇인가요?"})

'RAG(검색 보강 생성)는 정보 검색과 자연어 생성 기술을 결합한 모델로, 외부 데이터베이스에서 관련 정보를 검색한 후 이를 바탕으로 더 풍부하고 정확한 응답을 생성하는 방법입니다. 이를 통해 사용자의 질문에 대한 답변 품질을 향상시키고, 다양한 정보에 기반한 생성이 가능해집니다.'